In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import statsmodels.api as sm
import scipy
import scipy.stats

### Question 2

In [2]:
crsp = pd.read_csv('xrerfphrinihp8jn.csv')

In [3]:
crsp

,PERMNO,date,EXCHCD,TICKER,COMNAM,PERMCO,CUSIP,PRC,RET,vwretd,sprtrn
0,10107,20010131,3.0,MSFT,MICROSOFT CORP,8048,59491810,61.06250,0.407781,0.039573,0.034637
1,10107,20010228,3.0,MSFT,MICROSOFT CORP,8048,59491810,59.00000,-0.033777,-0.099084,-0.092291
2,10107,20010330,3.0,MSFT,MICROSOFT CORP,8048,59491810,54.68750,-0.073093,-0.070408,-0.064205
3,10107,20010430,3.0,MSFT,MICROSOFT CORP,8048,59491810,67.75000,0.238857,0.083834,0.076814
4,10107,20010531,3.0,MSFT,MICROSOFT CORP,8048,59491810,69.18000,0.021107,0.010442,0.005090
...,...,...,...,...,...,...,...,...,...,...,...
19805,93436,20180831,3.0,TSLA,TESLA INC,53453,88160R10,301.66000,0.011806,0.030233,0.030263
19806,93436,20180928,3.0,TSLA,TESLA INC,53453,88160R10,264.76999,-0.122290,0.000425,0.004294
19807,93436,20181031,3.0,TSLA,TESLA INC,53453,88160R10,337.32001,0.274011,-0.074045,-0.069403
19808,93436,20181130,3.0,TSLA,TESLA INC,53453,88160R10,350.48001,0.039013,0.018512,0.017859


#### Filtering & cleaning the data

In order to properly use the data we need to filter out the useble data and clean up unusable data from the data set. We started by checking for non-numeric values in the return column. Accoridng to the CRSP data guide, provided in the lecture videos, unusable data in the RET column ara marked down with the values 'B', 'C', '-66.0', '-77.0' or '-99.0'. The code below counts the number of cells in the RET column with for each of the stated values. Thereafter we replace these values, if they have been found in the data set, with the NaN (not a number) value. This allows us to convert the data under the RET column to floating numbers, which is done in the next line[7] of code.


In [4]:
print(sum(crsp.RET=='B'),sum(crsp.RET=='C'),sum(crsp.RET=='-66.0'),sum(crsp.RET=='-77.0'),sum(crsp.RET =='-99.0'))


35 27 0 0 0


In [5]:
crsp['RET'].replace(['C','B'],np.NaN,inplace=True)

In [6]:
print(sum(crsp.RET == 'B'),sum(crsp.RET == 'C'))

0 0


In [7]:
crsp['RET'] = pd.to_numeric(crsp['RET'],downcast='float')

In [8]:
crsp.dtypes

PERMNO      int64
date        int64
EXCHCD    float64
TICKER     object
COMNAM     object
PERMCO      int64
CUSIP      object
PRC       float64
RET       float32
vwretd    float64
sprtrn    float64
dtype: object

#### number of months

In order to filter out the stocks that are not present in the data set for all monthes we first count the number of total months over the whole time period. We do this by creating a variable called tot months which contains all unique dates in the data set. Here after we check the length of the variable 'totmonths' which givess us the number of uniqe dates, and thus the total amount of months. 


In [9]:
totmonths= crsp.date.unique()
nm=len(totmonths)
nm

216

* Here we duplicate the dataset for us to keep the orriginal data, if we need it later. 
* We add a variable 'months' to the unique data with a value of 1 for all rows. 
* here after we group buy 'TICKER', and sum all numeric variables, and we save this in a new dataframe named 'a'.

In [10]:
crsp2=crsp
crsp2['months']=1
a= crsp2.groupby(crsp2.TICKER).sum(numeric_only = True)

,PERMNO,date,EXCHCD,PERMCO,PRC,RET,vwretd,sprtrn,months
TICKER,,,,,,,,,
AAL,1282220,1229772031,183.0,1220610,2573.31100,0.605089,0.381465,0.358474,61
AAPL,3152088,4340666875,648.0,1512,38197.99376,6.389417,1.231263,0.830820,216
ADBE,16310160,4340666875,648.0,1830816,13227.87503,3.201410,1.231263,0.830820,216
ADI,13148136,4340666875,378.0,60912,9767.01630,1.881003,1.231263,0.830820,216
ADP,9643104,4340666875,462.0,4370760,13316.70101,1.706735,1.231263,0.830820,216
...,...,...,...,...,...,...,...,...,...
WLTW,3204612,726144480,108.0,1500984,5033.85998,0.251525,0.260980,0.222417,36
WSH,15577975,3514470745,175.0,7296450,6171.24000,1.730794,1.005926,0.648358,175
WYNN,17458935,3920353947,585.0,8505900,19028.98983,4.510199,1.625174,1.278225,195


we create a new dataframe named 'cm', which only includes data with a sum of months (number of months), equal to the total number of months over the complete time frame. This creates a datafram that only includes stocks that are present in the data set for all months. 

In [12]:
cm = a[a.months==nm]

,PERMNO,date,EXCHCD,PERMCO,PRC,RET,vwretd,sprtrn,months
TICKER,,,,,,,,,
AAPL,3152088,4340666875,648.0,1512,38197.99376,6.389417,1.231263,0.83082,216
ADBE,16310160,4340666875,648.0,1830816,13227.87503,3.201410,1.231263,0.83082,216
ADI,13148136,4340666875,378.0,60912,9767.01630,1.881003,1.231263,0.83082,216
ADP,9643104,4340666875,462.0,4370760,13316.70101,1.706735,1.231263,0.83082,216
ADSK,18496296,4340666875,648.0,1646568,10027.42102,4.460373,1.231263,0.83082,216
...,...,...,...,...,...,...,...,...,...
VRSN,18522648,4340666875,648.0,3443040,9836.80998,2.725839,1.231263,0.83082,216
VRTX,16576704,4340666875,648.0,2385072,12299.03639,2.937061,1.231263,0.83082,216
WDC,14338944,4340666875,374.0,1053864,8255.09240,5.319319,1.231263,0.83082,216


After this we store all the 'TICKER' values in under 'comp_firms' which have been included in data frame 'cm', which gives us all companies that should be included in the data frame that we will use for our analysis. 

In [58]:
comp_firms=cm.index.values[cm.months == nm]

Finally, we create a new data frame that includes data from the orriginal data frame 'crsp' for the stocks included in 'comp_firms', we name this new data frame 'crsp3'. This gives us a complete data set of all firms with data for the whole time period.

In [59]:
crsp3= crsp[crsp.TICKER.isin(comp_firms)]

In [57]:
crsp3

,PERMNO,date,EXCHCD,TICKER,COMNAM,PERMCO,CUSIP,PRC,RET,vwretd,sprtrn,months
0,10107,20010131,3.0,MSFT,MICROSOFT CORP,8048,59491810,61.0625,0.407781,0.039573,0.034637,1
1,10107,20010228,3.0,MSFT,MICROSOFT CORP,8048,59491810,59.0000,-0.033777,-0.099084,-0.092291,1
2,10107,20010330,3.0,MSFT,MICROSOFT CORP,8048,59491810,54.6875,-0.073093,-0.070408,-0.064205,1
3,10107,20010430,3.0,MSFT,MICROSOFT CORP,8048,59491810,67.7500,0.238857,0.083834,0.076814,1
4,10107,20010531,3.0,MSFT,MICROSOFT CORP,8048,59491810,69.1800,0.021107,0.010442,0.005090,1
...,...,...,...,...,...,...,...,...,...,...,...,...
18920,91556,20180831,3.0,ROST,ROSS STORES INC,7684,77829610,95.7800,0.095505,0.030233,0.030263,1
18921,91556,20180928,3.0,ROST,ROSS STORES INC,7684,77829610,99.1000,0.037012,0.000425,0.004294,1
18922,91556,20181031,3.0,ROST,ROSS STORES INC,7684,77829610,99.0000,-0.001009,-0.074045,-0.069403,1
18923,91556,20181130,3.0,ROST,ROSS STORES INC,7684,77829610,87.6000,-0.115152,0.018512,0.017859,1


#### cumulative returns

In the 'crsp3' dataframe we create a new column 'RET+1', which will be used to calculate cumulative returns. The code to do this gave us an error, which could be fixed by the code in the next line, whereafter we were able to create a new colum with the values from the 'RET' column plus 1.

/var/folders/n_/ckxw6r6s2kd_8cdnwww5y4hc0000gn/T/ipykernel_54512/2061118038.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp3['RET+1'] = crsp3['RET'] + 1


In [17]:
pd.options.mode.chained_assignment = None

In [18]:
crsp3['RET+1'] = crsp3['RET'] + 1

,PERMNO,date,EXCHCD,TICKER,COMNAM,PERMCO,CUSIP,PRC,RET,vwretd,sprtrn,months,RET+1
0,10107,20010131,3.0,MSFT,MICROSOFT CORP,8048,59491810,61.0625,0.407781,0.039573,0.034637,1,1.407781
1,10107,20010228,3.0,MSFT,MICROSOFT CORP,8048,59491810,59.0000,-0.033777,-0.099084,-0.092291,1,0.966223
2,10107,20010330,3.0,MSFT,MICROSOFT CORP,8048,59491810,54.6875,-0.073093,-0.070408,-0.064205,1,0.926907
3,10107,20010430,3.0,MSFT,MICROSOFT CORP,8048,59491810,67.7500,0.238857,0.083834,0.076814,1,1.238857
4,10107,20010531,3.0,MSFT,MICROSOFT CORP,8048,59491810,69.1800,0.021107,0.010442,0.005090,1,1.021107
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18920,91556,20180831,3.0,ROST,ROSS STORES INC,7684,77829610,95.7800,0.095505,0.030233,0.030263,1,1.095505
18921,91556,20180928,3.0,ROST,ROSS STORES INC,7684,77829610,99.1000,0.037012,0.000425,0.004294,1,1.037012
18922,91556,20181031,3.0,ROST,ROSS STORES INC,7684,77829610,99.0000,-0.001009,-0.074045,-0.069403,1,0.998991
18923,91556,20181130,3.0,ROST,ROSS STORES INC,7684,77829610,87.6000,-0.115152,0.018512,0.017859,1,0.884848


The code below creates a new column, named 'Cumulative_Return' with cumulative returns for every month from the start month up until the last month of the dataset. We need to use the groupby command and group by 'TICKER' so the cumulative return column is linked to the specifick stocks. Furthermore we add "['RET+1'].cumprod()" which lets us cumulate the return+1 values for all months for each ticker. 

In [20]:
crsp3['Cumulative_Return'] = crsp3.groupby('TICKER')['RET+1'].cumprod()

In [21]:
crsp3

,PERMNO,date,EXCHCD,TICKER,COMNAM,PERMCO,CUSIP,PRC,RET,vwretd,sprtrn,months,RET+1,Cumulative_Return
0,10107,20010131,3.0,MSFT,MICROSOFT CORP,8048,59491810,61.0625,0.407781,0.039573,0.034637,1,1.407781,1.407781
1,10107,20010228,3.0,MSFT,MICROSOFT CORP,8048,59491810,59.0000,-0.033777,-0.099084,-0.092291,1,0.966223,1.360230
2,10107,20010330,3.0,MSFT,MICROSOFT CORP,8048,59491810,54.6875,-0.073093,-0.070408,-0.064205,1,0.926907,1.260807
3,10107,20010430,3.0,MSFT,MICROSOFT CORP,8048,59491810,67.7500,0.238857,0.083834,0.076814,1,1.238857,1.561960
4,10107,20010531,3.0,MSFT,MICROSOFT CORP,8048,59491810,69.1800,0.021107,0.010442,0.005090,1,1.021107,1.594928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18920,91556,20180831,3.0,ROST,ROSS STORES INC,7684,77829610,95.7800,0.095505,0.030233,0.030263,1,1.095505,53.294594
18921,91556,20180928,3.0,ROST,ROSS STORES INC,7684,77829610,99.1000,0.037012,0.000425,0.004294,1,1.037012,55.267132
18922,91556,20181031,3.0,ROST,ROSS STORES INC,7684,77829610,99.0000,-0.001009,-0.074045,-0.069403,1,0.998991,55.211369
18923,91556,20181130,3.0,ROST,ROSS STORES INC,7684,77829610,87.6000,-0.115152,0.018512,0.017859,1,0.884848,48.853668


#### Date time variables

In the code below we create format the date variable to be a date-time variable instead of a integer. We had to added the part ".astype(str)" to first make a string out of the date variable, for the code to work. The format that we chose is YYYY-MM-DD.

In [22]:
crsp3.date = pd.to_datetime(crsp3['date'].astype(str), format='%Y%m%d')

In [23]:
crsp3

,PERMNO,date,EXCHCD,TICKER,COMNAM,PERMCO,CUSIP,PRC,RET,vwretd,sprtrn,months,RET+1,Cumulative_Return
0,10107,2001-01-31,3.0,MSFT,MICROSOFT CORP,8048,59491810,61.0625,0.407781,0.039573,0.034637,1,1.407781,1.407781
1,10107,2001-02-28,3.0,MSFT,MICROSOFT CORP,8048,59491810,59.0000,-0.033777,-0.099084,-0.092291,1,0.966223,1.360230
2,10107,2001-03-30,3.0,MSFT,MICROSOFT CORP,8048,59491810,54.6875,-0.073093,-0.070408,-0.064205,1,0.926907,1.260807
3,10107,2001-04-30,3.0,MSFT,MICROSOFT CORP,8048,59491810,67.7500,0.238857,0.083834,0.076814,1,1.238857,1.561960
4,10107,2001-05-31,3.0,MSFT,MICROSOFT CORP,8048,59491810,69.1800,0.021107,0.010442,0.005090,1,1.021107,1.594928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18920,91556,2018-08-31,3.0,ROST,ROSS STORES INC,7684,77829610,95.7800,0.095505,0.030233,0.030263,1,1.095505,53.294594
18921,91556,2018-09-28,3.0,ROST,ROSS STORES INC,7684,77829610,99.1000,0.037012,0.000425,0.004294,1,1.037012,55.267132
18922,91556,2018-10-31,3.0,ROST,ROSS STORES INC,7684,77829610,99.0000,-0.001009,-0.074045,-0.069403,1,0.998991,55.211369
18923,91556,2018-11-30,3.0,ROST,ROSS STORES INC,7684,77829610,87.6000,-0.115152,0.018512,0.017859,1,0.884848,48.853668


#### Finding the top performing stocks in 2001

after creating date-time variables we could easily filter the data from 2001 and place this in a new data frame, named 'year_2001'.

In [24]:
#create a new data frame which only includes data from the year 2001

year_2001 = crsp3[crsp3['date'].dt.year == 2001]

In [25]:
year_2001

,PERMNO,date,EXCHCD,TICKER,COMNAM,PERMCO,CUSIP,PRC,RET,vwretd,sprtrn,months,RET+1,Cumulative_Return
0,10107,2001-01-31,3.0,MSFT,MICROSOFT CORP,8048,59491810,61.0625,0.407781,0.039573,0.034637,1,1.407781,1.407781
1,10107,2001-02-28,3.0,MSFT,MICROSOFT CORP,8048,59491810,59.0000,-0.033777,-0.099084,-0.092291,1,0.966223,1.360230
2,10107,2001-03-30,3.0,MSFT,MICROSOFT CORP,8048,59491810,54.6875,-0.073093,-0.070408,-0.064205,1,0.926907,1.260807
3,10107,2001-04-30,3.0,MSFT,MICROSOFT CORP,8048,59491810,67.7500,0.238857,0.083834,0.076814,1,1.238857,1.561960
4,10107,2001-05-31,3.0,MSFT,MICROSOFT CORP,8048,59491810,69.1800,0.021107,0.010442,0.005090,1,1.021107,1.594928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18716,91556,2001-08-31,3.0,ROST,ROSS STORES INC,7684,77829610,29.3000,0.247555,-0.058980,-0.064108,1,1.247555,1.745426
18717,91556,2001-09-28,3.0,ROST,ROSS STORES INC,7684,77829610,29.2500,-0.001706,-0.091497,-0.081723,1,0.998294,1.742448
18718,91556,2001-10-31,3.0,ROST,ROSS STORES INC,7684,77829610,31.3000,0.070085,0.027847,0.018099,1,1.070085,1.864568
18719,91556,2001-11-30,3.0,ROST,ROSS STORES INC,7684,77829610,28.2100,-0.098722,0.078789,0.075176,1,0.901278,1.680494


In the code below we gather all values of 'Cumulative_Return' for the last month of 2001, this is done by adding ".last()" to the code. We store these values under 'dec2001_cumr'.

In [26]:
dec2001_cumr = year_2001.groupby('TICKER')['Cumulative_Return'].last()

Now we can sort all values in 'dec2001_cumr' from high to low using (ascending=False), and filter on the 10 highest valuess by using ".index.values[:10]". Then we store these 10 highest values as 'best10_2001' 

In [27]:
best10_2001 = dec2001_cumr.sort_values(ascending=False).index.values[:10]

Below the tickers of the 10 best performing stocks of 2001 can be found.

In [28]:
best10_2001

array(['NVDA', 'ATVI', 'WDC', 'EBAY', 'SYMC', 'ROST', 'MCHP', 'LRCX',
       'GILD', 'HAS'], dtype=object)

#### creating a data frame for dates between 2002 and 2018

* Here we filter data to have dates between january 2002 and december 2018, save the filtered data as 'crsp3_2002_2018'
* Then we drop the cumulative return column, because 2001 was included in these values, and we create a new cululative return column with data starting in 2002.

In [29]:
crsp3_2002_2018=crsp3[(crsp3['date']>= '2002-01-01') & (crsp3['date']<= '2018-12-31')]

crsp3_2002_2018 = crsp3_2002_2018.drop(columns=['Cumulative_Return'])
crsp3_2002_2018['Cum_Return_02'] = crsp3_2002_2018.groupby('TICKER')['RET+1'].cumprod()

In [30]:
crsp3_2002_2018

,PERMNO,date,EXCHCD,TICKER,COMNAM,PERMCO,CUSIP,PRC,RET,vwretd,sprtrn,months,RET+1,Cum_Return_02
12,10107,2002-01-31,3.0,MSFT,MICROSOFT CORP,8048,59491810,63.71,-0.038340,-0.015966,-0.015574,1,0.961660,0.961660
13,10107,2002-02-28,3.0,MSFT,MICROSOFT CORP,8048,59491810,58.34,-0.084288,-0.021700,-0.020766,1,0.915712,0.880604
14,10107,2002-03-28,3.0,MSFT,MICROSOFT CORP,8048,59491810,60.31,0.033768,0.044698,0.036739,1,1.033768,0.910340
15,10107,2002-04-30,3.0,MSFT,MICROSOFT CORP,8048,59491810,52.26,-0.133477,-0.049600,-0.061418,1,0.866523,0.788830
16,10107,2002-05-31,3.0,MSFT,MICROSOFT CORP,8048,59491810,50.91,-0.025832,-0.010510,-0.009081,1,0.974168,0.768453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18920,91556,2018-08-31,3.0,ROST,ROSS STORES INC,7684,77829610,95.78,0.095505,0.030233,0.030263,1,1.095505,27.850946
18921,91556,2018-09-28,3.0,ROST,ROSS STORES INC,7684,77829610,99.10,0.037012,0.000425,0.004294,1,1.037012,28.881765
18922,91556,2018-10-31,3.0,ROST,ROSS STORES INC,7684,77829610,99.00,-0.001009,-0.074045,-0.069403,1,0.998991,28.852623
18923,91556,2018-11-30,3.0,ROST,ROSS STORES INC,7684,77829610,87.60,-0.115152,0.018512,0.017859,1,0.884848,25.530186


,PERMNO,date,EXCHCD,TICKER,COMNAM,PERMCO,CUSIP,PRC,RET,vwretd,sprtrn,months,RET+1,Cum_Return_02,year,month,balance,investment_value
4937,48486,2002-01-31,3.0,LRCX,LAM RESH CORP,7013,51280710,23.26,0.001723,-0.015966,-0.015574,1,1.001723,1.001723,2002,1,0,1502.584595
4938,48486,2002-02-28,3.0,LRCX,LAM RESH CORP,7013,51280710,21.64,-0.069647,-0.021700,-0.020766,1,0.930353,0.931956,2002,2,0,1397.934082
4939,48486,2002-03-28,3.0,LRCX,LAM RESH CORP,7013,51280710,29.32,0.354898,0.044698,0.036739,1,1.354898,1.262705,2002,3,0,1894.057983
4940,48486,2002-04-30,3.0,LRCX,LAM RESH CORP,7013,51280710,25.66,-0.124829,-0.049600,-0.061418,1,0.875171,1.105083,2002,4,0,1657.624634
4941,48486,2002-05-31,3.0,LRCX,LAM RESH CORP,7013,51280710,22.74,-0.113796,-0.010510,-0.009081,1,0.886204,0.979329,2002,5,0,1468.993530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18920,91556,2018-08-31,3.0,ROST,ROSS STORES INC,7684,77829610,95.78,0.095505,0.030233,0.030263,1,1.095505,27.850946,2018,8,0,41776.417969
18921,91556,2018-09-28,3.0,ROST,ROSS STORES INC,7684,77829610,99.10,0.037012,0.000425,0.004294,1,1.037012,28.881765,2018,9,0,43322.648438
18922,91556,2018-10-31,3.0,ROST,ROSS STORES INC,7684,77829610,99.00,-0.001009,-0.074045,-0.069403,1,0.998991,28.852623,2018,10,0,43278.933594
18923,91556,2018-11-30,3.0,ROST,ROSS STORES INC,7684,77829610,87.60,-0.115152,0.018512,0.017859,1,0.884848,25.530186,2018,11,0,38295.277344


#### Creating a data frame for december

Here we make a new data frame for only the last mont of 2018

In [31]:
crsp3_2002_2018['year']=crsp3_2002_2018['date'].dt.year
crsp3_2002_2018['month']=crsp3_2002_2018['date'].dt.month

In [80]:
q= crsp3_2002_2018[crsp3_2002_2018.TICKER.isin(best10_2001)]

q['investment_value'] = q['Cum_Return_02']*1500



In [76]:
q['year']=q['date'].dt.year
q['month']=q['date'].dt.month

q = q[q['date'].dt.year == 2018]
q = q[q['date'].dt.month == 12]

In [77]:
q

,PERMNO,date,EXCHCD,TICKER,COMNAM,PERMCO,CUSIP,PRC,RET,vwretd,sprtrn,months,RET+1,Cum_Return_02,year,month,balance,investment_value
5140,48486,2018-12-31,3.0,LRCX,LAM RESH CORP,7013,51280710,136.170,-0.125446,-0.08989,-0.091777,1,0.874554,6.304788,2018,12,0,9457.182617
5356,52978,2018-12-31,3.0,HAS,HASBRO INC,20887,41805610,81.250,-0.107143,-0.08989,-0.091777,1,0.892857,7.448472,2018,12,0,11172.708984
7084,66384,2018-12-31,3.0,WDC,WESTERN DIGITAL CORP,4879,95810210,36.970,-0.174488,-0.08989,-0.091777,1,0.825512,6.952835,2018,12,0,10429.251953
7850,75607,2018-12-31,3.0,SYMC,SYMANTEC CORP,10224,66877110,18.895,-0.145409,-0.08989,-0.091777,1,0.854591,3.107772,2018,12,0,4661.658203
9794,77274,2018-12-31,3.0,GILD,GILEAD SCIENCES INC,11300,37555810,62.550,-0.122602,-0.08989,-0.091777,1,0.877398,16.755398,2018,12,0,25133.097656
10658,78987,2018-12-31,3.0,MCHP,MICROCHIP TECHNOLOGY INC,12027,59501710,71.920,-0.041067,-0.08989,-0.091777,1,0.958933,4.482806,2018,12,0,6724.209473
11166,79678,2018-12-31,3.0,ATVI,ACTIVISION BLIZZARD INC,12499,00507V10,46.570,-0.066359,-0.08989,-0.091777,1,0.933641,15.740643,2018,12,0,23610.962891
14823,86356,2018-12-31,3.0,EBAY,EBAY INC,16285,27864210,28.070,-0.059632,-0.08989,-0.091777,1,0.940368,4.055719,2018,12,0,6083.578125
15039,86580,2018-12-31,3.0,NVDA,NVIDIA CORP,16382,67066G10,133.500,-0.183136,-0.08989,-0.091777,1,0.816864,6.470007,2018,12,0,9705.010742
18924,91556,2018-12-31,3.0,ROST,ROSS STORES INC,7684,77829610,83.200,-0.047660,-0.08989,-0.091777,1,0.952340,24.313417,2018,12,0,36470.125000


In [78]:
sum(q['investment_value'])

143447.78564453125

filter the data from dataframe 'crsp3_2002_2018' to only include data for December 2018. 
This wil allow us to directly find the cumulative return of for the last month of the investment period
 data from the dataframe 'crsp3_2002_2018'

In [35]:
december_2018_cum = crsp3_2002_2018[crsp3_2002_2018['date'].dt.year == 2018]
december_2018_cum = december_2018_cum[december_2018_cum['date'].dt.month == 12]

In [36]:
december_2018_cum

,PERMNO,date,EXCHCD,TICKER,COMNAM,PERMCO,CUSIP,PRC,RET,vwretd,sprtrn,months,RET+1,Cum_Return_02,year,month
215,10107,2018-12-31,3.0,MSFT,MICROSOFT CORP,8048,59491810,101.57000,-0.084047,-0.08989,-0.091777,1,0.915953,4.698978,2018,12
431,10696,2018-12-31,3.0,FISV,FISERV INC,8598,33773810,73.49000,-0.071275,-0.08989,-0.091777,1,0.928725,6.946142,2018,12
698,10909,2018-12-31,3.0,CERN,CERNER CORP,8786,15678210,52.44000,-0.094457,-0.08989,-0.091777,1,0.905543,8.402165,2018,12
1130,11552,2018-12-31,3.0,CELG,CELGENE CORP,9374,15102010,64.09000,-0.112573,-0.08989,-0.091777,1,0.887427,16.062550,2018,12
1346,11618,2018-12-31,3.0,FAST,FASTENAL CO,9440,31190010,52.29000,-0.117617,-0.08989,-0.091777,1,0.882383,8.572522,2018,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16278,88352,2018-12-31,3.0,ISRG,INTUITIVE SURGICAL INC,37727,46120E60,478.92001,-0.097858,-0.08989,-0.091777,1,0.902142,71.622681,2018,12
16494,88362,2018-12-31,3.0,NTES,NETEASE INC,37733,64110W10,235.37000,0.036553,-0.08989,-0.091777,1,1.036553,1608.175171,2018,12
16710,88446,2018-12-31,3.0,ILMN,ILLUMINA INC,37938,45232710,299.92999,-0.111319,-0.08989,-0.091777,1,0.888681,51.008308,2018,12
17020,88860,2018-12-31,3.0,ALGN,ALIGN TECHNOLOGY INC,41154,01625510,209.42999,-0.088999,-0.08989,-0.091777,1,0.911001,46.539974,2018,12


create an empty list and loop to check for all of the 10 stocks in 'best10_2001' what their cumulative return was
in december 2018. 

In [37]:
cumulative_returns = {}
for ticker in best10_2001:
    if ticker in december_2018_cum['TICKER'].values:
        ticker_data = december_2018_cum[december_2018_cum['TICKER'] == ticker]
        cumulative_return = ticker_data['Cum_Return_02'] 
        cumulative_returns[ticker] = cumulative_return.values

In [38]:

investment_value = (15000 / 10)  
Val_2001inv_in2018 = {ticker: investment_value*cumulative_return for ticker, cumulative_return in cumulative_returns.items()}


In [39]:
Val_2001inv_in2018

{'NVDA': array([9705.011], dtype=float32),
 'ATVI': array([23610.963], dtype=float32),
 'WDC': array([10429.252], dtype=float32),
 'EBAY': array([6083.578], dtype=float32),
 'SYMC': array([4661.658], dtype=float32),
 'ROST': array([36470.125], dtype=float32),
 'MCHP': array([6724.2095], dtype=float32),
 'LRCX': array([9457.183], dtype=float32),
 'GILD': array([25133.098], dtype=float32),
 'HAS': array([11172.709], dtype=float32)}

sum the investment value at the end of 2018 for all top 10 stocks from 2001

In [40]:
total_investment_value_2001 = sum(Val_2001inv_in2018.values())


print(total_investment_value_2001)

[143447.78]


#### total value of invesstmentt from 2001

The code above gives us the total value of the investment from 2001 in the top 10 stocks from 2001, which has a value of 143447.78 dollar

#### Finding the top 2 stocks for each month

* Here we make new data frame which groups the data on month and year, and gives the 2 highest return values for every group, whit is done by adding '['RET'].nlargest(2)' at the end of the string.
* Then we index the top two in the two best performing stocks and save this index as 'top2_index'


In [41]:
top_2_per_month=crsp3_2002_2018.groupby(['year', 'month'])['RET'].nlargest(2)

top2_index = top_2_per_month.index.levels[2]

year  month       
2002  1      13530    0.311460
             10023    0.247769
      2      13315    0.152134
             1144     0.118517
      3      16509    0.379009
                        ...   
2018  10     4035     0.038128
      11     7849     0.222314
             6435     0.169687
      12     12030    0.040913
             16494    0.036553
Name: RET, Length: 408, dtype: float32

Int64Index([   81,   513,   516,   556,   587,   616,   965,   980,   999,
             1059,
            ...
            16910, 16928, 16934, 16958, 17000, 17006, 17013, 18746, 18872,
            18875],
           dtype='int64', length=408)

* Here after we make a data frame named 'top_return_df', which includes 'year', 'month', 'date', 'TICKER' and 'RET', found in crsp3_2002_2018 for all index values from top2_index.
* then we sort the values by dates and store this as a new data frame named 'top_return_sorted_date'

In [45]:
top_return_df = crsp3_2002_2018.loc[top2_index, ['year', 'month', 'date', 'TICKER', 'RET']]

top_return_sorted_date = top_return_df.sort_values(by='date')


,year,month,date,TICKER,RET
81,2007,10,2007-10-31,MSFT,0.249491
513,2003,7,2003-07-31,CERN,0.388158
516,2003,10,2003-10-31,CERN,0.372530
556,2007,2,2007-02-28,CERN,0.159804
587,2009,9,2009-09-30,CERN,0.212121
...,...,...,...,...,...
17006,2017,10,2017-10-31,ALGN,0.282976
17013,2018,5,2018-05-31,ALGN,0.328597
18746,2004,2,2004-02-27,ROST,0.152334
18872,2014,8,2014-08-29,ROST,0.171118


In [48]:
top_return_sorted_date

,year,month,date,TICKER,RET
10023,2002,1,2002-01-31,SBUX,0.247769
13530,2002,1,2002-01-31,AMZN,0.311460
13315,2002,2,2002-02-28,TTWO,0.152134
1144,2002,2,2002-02-28,FAST,0.118517
16509,2002,3,2002-03-28,ILMN,0.379009
...,...,...,...,...,...
4035,2018,10,2018-10-31,XEL,0.038128
6435,2018,11,2018-11-30,AMD,0.169687
7849,2018,11,2018-11-30,SYMC,0.222314
12030,2018,12,2018-12-31,DLTR,0.040913


,PERMNO,date,EXCHCD,TICKER,COMNAM,PERMCO,CUSIP,PRC,RET,vwretd,sprtrn,months,RET+1,Cum_Return_02,year,month,balance
12,10107,2002-01-31,3.0,MSFT,MICROSOFT CORP,8048,59491810,63.71,-0.038340,-0.015966,-0.015574,1,0.961660,0.961660,2002,1,0
13,10107,2002-02-28,3.0,MSFT,MICROSOFT CORP,8048,59491810,58.34,-0.084288,-0.021700,-0.020766,1,0.915712,0.880604,2002,2,0
14,10107,2002-03-28,3.0,MSFT,MICROSOFT CORP,8048,59491810,60.31,0.033768,0.044698,0.036739,1,1.033768,0.910340,2002,3,0
15,10107,2002-04-30,3.0,MSFT,MICROSOFT CORP,8048,59491810,52.26,-0.133477,-0.049600,-0.061418,1,0.866523,0.788830,2002,4,0
16,10107,2002-05-31,3.0,MSFT,MICROSOFT CORP,8048,59491810,50.91,-0.025832,-0.010510,-0.009081,1,0.974168,0.768453,2002,5,0
17,10107,2002-06-28,3.0,MSFT,MICROSOFT CORP,8048,59491810,54.70,0.074445,-0.070259,-0.072465,1,1.074445,0.825661,2002,6,0
18,10107,2002-07-31,3.0,MSFT,MICROSOFT CORP,8048,59491810,47.98,-0.122852,-0.081125,-0.078995,1,0.877148,0.724227,2002,7,0
19,10107,2002-08-30,3.0,MSFT,MICROSOFT CORP,8048,59491810,49.08,0.022926,0.007949,0.004881,1,1.022926,0.740830,2002,8,0
20,10107,2002-09-30,3.0,MSFT,MICROSOFT CORP,8048,59491810,43.74,-0.108802,-0.099923,-0.110013,1,0.891198,0.660227,2002,9,0
21,10107,2002-10-31,3.0,MSFT,MICROSOFT CORP,8048,59491810,53.47,0.222451,0.074940,0.086436,1,1.222451,0.807095,2002,10,0
